In [1]:
 from __future__ import print_function
import tensorflow as tf
import os
import import_ipynb
from skimage.transform import resize
from skimage.io import imsave
from data import load_test_data , load_train_data

import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose,Dense , Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.activations import softmax
 
 
 
 
K.set_image_data_format('channels_last')  # TF dimension ordering in this code
 
img_rows = 96
img_cols = 96
 
smooth = 1.
 
 
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    
 
 
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
 
 
def get_unet():
    inputs = Input((img_rows, img_cols, 1))
    print("++++++++++++++++++++++++++++++++++++++++++++++new Line")
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(0.5)(pool1)
        
        
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)


    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)
 
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)
    
    
    print(tf.shape(conv4))
    #split
    td=conv4
    
    tx=softmax(td,axis=[1,2])
    q=1.2
    
    
    
    safe_x = K.maximum(tx,1e-6)
    new_p = K.switch(K.equal(q,1.), K.log(safe_x),(K.pow(safe_x,q))/(q-1))
    new_p = -new_p #-p^q/(q-1)
 
 
    new_p= 1+new_p
    print("new_p",new_p.shape)
    xyz=K.max(new_p,axis=[1,2],keepdims=True)
    print("xyz=",xyz.shape)
   
    
    print("xyz=",xyz.shape)
    weights=1-(new_p/xyz)
    print("weights",weights.shape)
    improved_p=td*weights
    print("improved_p=",improved_p.shape)
    improved_p=K.sum(improved_p,axis=[1,2])/144
    print("improved_pv2=",type(improved_p),improved_p.shape)
    
#     n=tx.shape[0]
#     temp=tf.zeros([0,0],tf.float32)
    
#     for i in range(n):
#         print("=========inside loop",i)
#         safe_x = K.maximum(tx[i],1e-6)
#         new_p = K.switch(K.equal(q,1.), K.log(safe_x),(K.pow(safe_x,q))/(q-1))
#         new_p = -new_p #-p^q/(q-1)
    
    
#         new_p= 1+new_p
#         weights=1-(new_p/K.max(new_p))
    
#         improved_p=td[i]*weights
        
#         temp=tf.concat([temp,(K.sum(improved_p)/144)],0)
    
    
    
    
#     dense_nodes=K.reshape(K.variable(temp),shape=(1,256))
#     print(dense_nodes.shape)
    ####
#     den=[]
#     for i in range(n):
#         den.append(tf.reduce_sum(before_gap[i])/144)
    
    
#     dense_nodes=tf.Variable(den)
#     dense_nodes=tf.reshape(dense_nodes,shape=(1,256))
    
#     dense_nodes=K.sum(before_gap,axis=1)/144.0
#     print(dense_nodes.shape)
#     dense_nodes=K.reshape(before_gap,shape=(1,256))
    
    
#     print("temp type=",type(temp[0]))
#     dense_nodes = tf. convert_to_tensor(improved_p)
#     print("dense node=",dense_nodes)
#     xy=tf.reshape(dense_nodes,(1,256))
#     print("xy=",xy)
    #improved_p=tf.reshape(improved_p,(1,256))
    dense1=Dense(64,activation='relu')(improved_p)
    print("dense1",dense1)
    dense2=Dense(32,activation='relu')(dense1)
    print("below dense",dense2)
    dense3=Dense(1,activation='softmax')(dense2)
    print("below dense",dense3)
    
    
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = Dropout(0.5)(conv5)
    
    print("conv5=",conv5)
 
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = Dropout(0.5)(conv6)
 
    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = Dropout(0.5)(conv7)
 
    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = Dropout(0.5)(conv8)
 
    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = Dropout(0.5)(conv9)
 
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    print("conv10=", conv10) 
 
    model = Model(inputs=[inputs], outputs=[conv10,dense3])
 
    model.compile(optimizer=Adam(lr=1e-5), loss=[dice_coef_loss,
                                                 tf.keras.losses.BinaryCrossentropy()],
                  metrics=[dice_coef,tf.keras.metrics.BinaryCrossentropy()])
 
    return model
 
 
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)
 
    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p
 
 
def train_and_predict():
    imgs_train,imgs_mask_train,imgs_value_train=load_train_data()
 
    imgs_train = preprocess(imgs_train)
    imgs_mask_train = preprocess(imgs_mask_train)
 
    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization
 
    imgs_train -= mean
    imgs_train /= std
 
    imgs_mask_train = imgs_mask_train.astype('float32')
    imgs_mask_train /= 255.  # scale masks to [0, 1]
 
    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_unet()
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss' , mode = 'min' , verbose = 1 , save_best_only=True )
 
    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    model.fit(imgs_train, y=[imgs_mask_train,imgs_value_train], batch_size=32, epochs= 20, verbose=1, shuffle=True,
              validation_split=0.2 ,  callbacks=[model_checkpoint])
    
    print('-'*30)
    print('Loading and preprocessing test data...')
    print('-'*30)
    imgs_test, imgs_id_test = load_test_data()
    imgs_test = preprocess(imgs_test)

    imgs_test = imgs_test.astype('float32')
    imgs_test -= mean
    imgs_test /= std

    print('-'*30)
    print('Loading saved weights...')
    print('-'*30)
    model.load_weights('weights.h5')

    print('-'*30)
    print('Predicting masks on test data...')
    print('-'*30)
    n_imgs_test = imgs_test.shape[0]
    #for i in range(n_imgs_test):

    #worked for 7 vs 1
    #imgs_mask_test, imgs_value_test= model.predict(np.expand_dims(imgs_test[0] , axis = 0), verbose=1)


    imgs_mask_test, imgs_value_test= model.predict(imgs_test, verbose=1)
    print("mask final img 1" , imgs_mask_test)
    np.save('imgs_mask_test.npy', imgs_mask_test)

    print('-' * 30)
    print('Saving predicted masks to files...')
    print('-' * 30)
    pred_dir = 'preds'
    
    # print(imgs_mask_test.shape , imgs_value_test.shape)
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
    for image, image_id in zip(imgs_mask_test, imgs_id_test):
        image = (image[:, :, 0] * 255.).astype(np.uint8)
        imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)


 
     
if __name__ == '__main__':
    train_and_predict()

importing Jupyter notebook from data.ipynb
------------------------------
Creating and compiling model...
------------------------------
++++++++++++++++++++++++++++++++++++++++++++++new Line
Tensor("Shape:0", shape=(4,), dtype=int32)
new_p (None, 12, 12, 256)
xyz= (None, 1, 1, 256)
xyz= (None, 1, 1, 256)
weights (None, 12, 12, 256)
improved_p= (None, 12, 12, 256)
improved_pv2= <class 'tensorflow.python.framework.ops.Tensor'> (None, 256)
dense1 Tensor("dense/Relu:0", shape=(None, 64), dtype=float32)
below dense Tensor("dense_1/Relu:0", shape=(None, 32), dtype=float32)
below dense Tensor("dense_2/Softmax:0", shape=(None, 1), dtype=float32)
conv5= Tensor("dropout_4/cond/Identity:0", shape=(None, 6, 6, 512), dtype=float32)
conv10= Tensor("conv2d_18/Sigmoid:0", shape=(None, 96, 96, 1), dtype=float32)
------------------------------
Fitting model...
------------------------------
Epoch 1/20
141/141 [==============================] - ETA: 0s - loss: 8.9093 - conv2d_18_loss: -0.0244 - dense_2_

Epoch 10/20
141/141 [==============================] - ETA: 0s - loss: 8.5540 - conv2d_18_loss: -0.3797 - dense_2_loss: 8.9337 - conv2d_18_dice_coef: 0.3798 - conv2d_18_binary_crossentropy: 0.2407 - dense_2_dice_coef: 0.5907 - dense_2_binary_crossentropy: 8.9337
Epoch 00010: val_loss improved from 8.91897 to 8.91485, saving model to weights.h5
141/141 [==============================] - 680s 5s/step - loss: 8.5540 - conv2d_18_loss: -0.3797 - dense_2_loss: 8.9337 - conv2d_18_dice_coef: 0.3798 - conv2d_18_binary_crossentropy: 0.2407 - dense_2_dice_coef: 0.5907 - dense_2_binary_crossentropy: 8.9337 - val_loss: 8.9148 - val_conv2d_18_loss: -0.1643 - val_dense_2_loss: 9.0792 - val_conv2d_18_dice_coef: 0.1624 - val_conv2d_18_binary_crossentropy: 0.6890 - val_dense_2_dice_coef: 0.5789 - val_dense_2_binary_crossentropy: 9.0792
Epoch 11/20
141/141 [==============================] - ETA: 0s - loss: 8.5428 - conv2d_18_loss: -0.3909 - dense_2_loss: 8.9337 - conv2d_18_dice_coef: 0.3909 - conv2d_18_b

Epoch 20/20
141/141 [==============================] - ETA: 0s - loss: 8.4590 - conv2d_18_loss: -0.4748 - dense_2_loss: 8.9337 - conv2d_18_dice_coef: 0.4747 - conv2d_18_binary_crossentropy: 0.2483 - dense_2_dice_coef: 0.5903 - dense_2_binary_crossentropy: 8.9337
Epoch 00020: val_loss did not improve from 8.75854
141/141 [==============================] - 740s 5s/step - loss: 8.4590 - conv2d_18_loss: -0.4748 - dense_2_loss: 8.9337 - conv2d_18_dice_coef: 0.4747 - conv2d_18_binary_crossentropy: 0.2483 - dense_2_dice_coef: 0.5903 - dense_2_binary_crossentropy: 8.9337 - val_loss: 8.7808 - val_conv2d_18_loss: -0.2984 - val_dense_2_loss: 9.0792 - val_conv2d_18_dice_coef: 0.2948 - val_conv2d_18_binary_crossentropy: 0.4596 - val_dense_2_dice_coef: 0.5789 - val_dense_2_binary_crossentropy: 9.0792
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
--------------

/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/2007_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/3899_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/331_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/3292_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/2125_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/5028_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/2313_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/sit

/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/4399_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/2999_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/506_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/3785_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/5253_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/5044_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/2250_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/sit

/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/4919_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/2444_pred.png is a low contrast image
/home/sh1vansh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:258: UserWarning: preds/1090_pred.png is a low contrast image
